In [1]:
# Task1 Hint: (with sample code for the SIFT detector)
# Initialize SIFT detector, detect keypoints, store and show SIFT keypoints of original image in a Numpy array
# Define parameters for SIFT initializations such that we find only 10% of keypoints
import cv2
import matplotlib.pyplot as plt
import copy

class SiftDetector():
    def __init__(self, norm="L2", params=None):
        self.detector=self.get_detector(params)
        self.norm=norm

    def get_detector(self, params):
        if params is None:
            params={}
            params["n_features"]=0
            params["n_octave_layers"]=3
            params["contrast_threshold"]=0.03
            params["edge_threshold"]=10
            params["sigma"]=1.6

        detector = cv2.xfeatures2d.SIFT_create(
                nfeatures=params["n_features"],
                nOctaveLayers=params["n_octave_layers"],
                contrastThreshold=params["contrast_threshold"],
                edgeThreshold=params["edge_threshold"],
                sigma=params["sigma"])

        return detector



# Task2 Hint:
# Upscale the image, compute SIFT features for rescaled image
# Apply BFMatcher with defined params and ratio test to obtain good matches, and then select and draw best 5 matches

# Task3 Hint: (with sampe code for the rotation)
# Rotate the image and compute SIFT features for rotated image
# Apply BFMatcher with defined params and ratio test to obtain good matches, and then select and draw best 5 matches
import math
import numpy as np
import sys

# image: image to rotate
# x:     x-coordinate of point we wish to rotate around
# y:     y-coordinate of point we wish to rotate around
# angle: degrees to rotate image by
#
# Returns a rotated copy of the original image
def rotate(image, x, y, angle):
    rot_matrix = cv2.getRotationMatrix2D((x, y), angle, 1.0)
    h, w = image.shape[:2]

    return cv2.warpAffine(image, rot_matrix, (w, h))

# Get coordinates of center point.
#
# image:  Image that will be rotated
# return: (x, y) coordinates of point at center of image
def get_img_center(image):
    height, width = image.shape[:2]
    center = height // 2, width // 2

    return center


In [2]:
img_gray = cv2.imread('COMP9517_20T2_Lab2_Image.jpg', 0)
img = cv2.imread('COMP9517_20T2_Lab2_Image.jpg')
# img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
# img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [3]:
A = SiftDetector()
detector_1 = A.get_detector(params=None)

In [4]:
kp1, des1 = detector_1.detectAndCompute(img_gray, None)

In [5]:
img_kp = copy.deepcopy(img)
img_kp = cv2.drawKeypoints(img, kp1, img_kp, color=(255,0,255))

In [76]:
parameters = {'n_features':600, 'n_octave_layers':3, 'contrast_threshold':0.08, 'edge_threshold':10, 'sigma':1.6}
B = SiftDetector()
detector_2 = B.get_detector(parameters)
kp2, des2 = detector_2.detectAndCompute(img_gray, None)
img_kp_2 = copy.deepcopy(img)
img_kp_2 = cv2.drawKeypoints(img, kp2, img_kp_2, color=(255,0,255))
# cv2.imshow('output', img_kp_2)
# cv2.waitKey(10)
# cv2.destroyAllWindows()

In [77]:
#nfeatures：特征点数目（算法对检测出的特征点排名，返回最好的nfeatures个特征点）。
#nOctaveLayers：金字塔中每组的层数（算法中会自己计算这个值，后面会介绍）。
#contrastThreshold：过滤掉较差的特征点的对阈值。contrastThreshold越大，返回的特征点越少。
#edgeThreshold：过滤掉边缘效应的阈值。edgeThreshold越大，特征点越多（被过滤掉的越少）。
#sigma：金字塔第0层图像高斯滤波系数，也就是σ。

#nfeatures指定最终返回的特征点数量，并不影响SIFT特征检测的结果
#参数nOctaveLayers和sigma主要影响图像高斯金字塔的构成，
#contrastThreshold和edgeThreshold则影响在DOG中寻找极值点的过程与结果。

In [78]:
#task2

In [79]:
height, width = int(img.shape[0]*1.15), int(img.shape[1]*1.15)


In [80]:
img_gray_scale = cv2.resize(img_gray, dsize=(width, height), interpolation=cv2.INTER_NEAREST)
img_scale = cv2.resize(img, dsize=(width, height), interpolation=cv2.INTER_NEAREST)

In [81]:
kp_scale, des_scale = detector_2.detectAndCompute(img_gray_scale, None)
img_scale_2 = copy.deepcopy(img_scale)
img_scale_2 = cv2.drawKeypoints(img_scale, kp_scale, img_scale_2, color=(255,0,255))

In [82]:
#nearest-neighbour distance ratio
flann = cv2.FlannBasedMatcher()
matches_scale = flann.knnMatch(des2, des_scale, k=2)
ratio = 0.7
best_match_scale = []
for i, (m, n) in enumerate(matches_scale):
    if m.distance < ratio * n.distance:
        best_match_scale.append(m)
best_match_scale = sorted(best_match_scale, key = lambda x: x.distance)
sorted_match = []
for m in best_match_scale:
    sorted_match.append([m])
img3 = cv2.drawMatchesKnn(img_kp_2,kp2,img_scale_2,kp_scale,sorted_match[:5], None,flags=2)

In [83]:
cv2.imshow('output', img3)
cv2.waitKey(10)
cv2.destroyAllWindows()

In [84]:
#task 3

In [85]:
center = get_img_center(img)

In [86]:
def rotate_bound(image, angle):
    # grab the dimensions of the image and then determine the
    # center
    (h, w) = image.shape[:2]
    (cX, cY) = (w // 2, h // 2)
 
    # grab the rotation matrix (applying the negative of the
    # angle to rotate clockwise), then grab the sine and cosine
    # (i.e., the rotation components of the matrix)
    M = cv2.getRotationMatrix2D((cX, cY), -angle, 1.0)
    cos = np.abs(M[0, 0])
    sin = np.abs(M[0, 1])
 
    # compute the new bounding dimensions of the image
    nW = int((h * sin) + (w * cos))
    nH = int((h * cos) + (w * sin))
 
    # adjust the rotation matrix to take into account translation
    M[0, 2] += (nW / 2) - cX
    M[1, 2] += (nH / 2) - cY
 
    # perform the actual rotation and return the image
    return cv2.warpAffine(image, M, (nW, nH))

In [87]:
img_gray_rotate = rotate_bound(img_gray, 60)
img_rotate = rotate_bound(img, 60)

In [88]:
kp_rotate, des_rotate = detector_2.detectAndCompute(img_gray_rotate, None)
img_rotate_4 = copy.deepcopy(img_rotate)
img_rotate_4 = cv2.drawKeypoints(img_rotate_4, kp_rotate, img_rotate_4, color=(255,0,255))

In [89]:
#nearest-neighbour distance ratio
matches_rotate = flann.knnMatch(des2, des_rotate, k=2)
ratio = 0.5
best_match_rotate = []
for i, (m, n) in enumerate(matches_rotate):
    if m.distance < ratio * n.distance:
        best_match_rotate.append(m)
best_match_rotate = sorted(best_match_rotate, key = lambda x: x.distance)
sorted_match_rotate = []
for m in best_match_rotate:
    sorted_match_rotate.append([m])
img_rotate_output = cv2.drawMatchesKnn(img_kp_2,kp2,img_rotate_4,kp_rotate,sorted_match_rotate[:5], None,flags=2)
cv2.imshow('output', img_rotate_output)
cv2.waitKey(10)
cv2.destroyAllWindows()

In [90]:
for i in best_match_rotate[:5]:
    print(i.distance)

6.633249759674072
7.211102485656738
9.05538558959961
10.198039054870605
11.357816696166992
